# A. State preparation with Jaynes-Cummings controls

In [1]:
# ruff: noqa
from feedback_grape.grape import optimize_pulse_parameterized
from feedback_grape.utils.operators import sigmap, sigmam, create, destroy
from feedback_grape.utils.states import basis, fock
from feedback_grape.utils.tensor import tensor
import jax.numpy as jnp

As a preliminary step, we consider state preparation
of a target state starting from a pure state. In addition,
we assume that any coupling to an external environment
is negligible and that the parametrized controls can be
implemented perfectly.

Here no feedback is required, we are just testing the parameterized gates setup.

As a first example, we consider the state preparation
of a cavity resonantly coupled to an externally driven
qubit

Here, we consider a particular sequence of
parametrized unitary gates originally introduced by Law
and Eberly

In [2]:
N_cav = 4

In [3]:
def qubit_unitary(alpha):
    return jnp.exp(-1j * (alpha * sigmap() + alpha.conj() * sigmam(2)) / 2)

In [4]:
def qubit_cavity_unitary(beta):
    return jnp.exp(
        -1j
        * (
            beta * (destroy(N_cav) @ sigmap())
            + beta.conj() * (create(N_cav) @ sigmam(2)) / 2
        )
    )

In their groundbreaking work, Law and Eberly have
shown that any arbitrary superposition of Fock states with
maximal excitation number N can be prepared out of the
ground state in a sequence of N such interleaved gates,
also providing an algorithm to find the correct angles and
interaction durations

## First target is the state $ | 1 ⟩ + | 3 ⟩ $ 

In [5]:
time_steps = 3  # corressponds to maximal excitation number of an arbitrary Fock State Superposition

In [6]:
psi0 = tensor(basis(2), basis(N_cav))
psi_target = fock(1, N_cav) + fock(3, N_cav)

In [7]:
result = optimize_pulse_parameterized(
    U_0=psi0,
    C_target=psi_target,
    feedback=False,
    parameterized_gates=[qubit_unitary, qubit_cavity_unitary],
    mode="nn",
    goal="fidelity",
    num_time_steps=time_steps,
    optimizer="adam",
    max_iter=1000,
    convergence_threshold=1e-6,
    type="state",
)